In [ ]:
# Hyperparameter tuning and optimization
import optuna
from ray.tune.search.optuna import OptunaSearch
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.pytorch_lightning import TuneReportCallback, TuneReportCheckpointCallback

# PyTorch Lightning and callbacks
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, Callback, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

# Metrics
from torchmetrics import MeanAbsoluteError, MeanAbsolutePercentageError, MetricCollection

# Darts (Time series forecasting)
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import DLinearModel, LightGBMModel, BlockRNNModel, TiDEModel

# Data handling and preprocessing
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorboard

# System utilities
import os

Load Data / Spilt Data

In [ ]:
# 步骤1: 加载CSV文件
df = pd.read_csv('../DataSet/EDvisitfileC.csv', encoding='ISO-8859-1')

# 确保'date'列是DateTime类型
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# 分割数据集为训练集、验证集和测试集（假设您已经根据时间排序）
train_end = 3237            #L: 3362, T:3372, Ka:3208, Ke:3274, Y:2557, C: 3237
val_end = 3602              #L: 3727, T:3737, Ka:3573, Ke:3639, Y:2822, C: 3602

# Split the DataFrame
train_df = df.iloc[:train_end]
val_df = df.iloc[train_end:val_end]
test_df = df.iloc[val_end:]

# 步骤2: 使用MinMaxScaler缩放数据
# 定义并拟合scaler
scaler = MinMaxScaler()
scaler.fit(train_df[['No']])  # 只用训练数据拟合scaler

# 缩放训练集和验证集
train_df.loc[:, 'No_scaled'] = scaler.transform(train_df[['No']])
val_df.loc[:, 'No_scaled'] = scaler.transform(val_df[['No']])
test_df.loc[:, 'No_scaled'] = scaler.transform(test_df[['No']])  # 用相同的scaler转换测试集以避免数据泄露

# 转换为TimeSeries对象
train_series = TimeSeries.from_dataframe(train_df, value_cols='No_scaled')
val_series = TimeSeries.from_dataframe(val_df, value_cols='No_scaled')
test_series = TimeSeries.from_dataframe(test_df, value_cols='No_scaled')

# 原始数据转换为TimeSeries对象，如果需要
train_series_origin = TimeSeries.from_dataframe(train_df, value_cols='No')
val_series_origin = TimeSeries.from_dataframe(val_df, value_cols='No')
test_series_origin = TimeSeries.from_dataframe(test_df, value_cols='No')

# 选择需要的列创建多变量时间序列(都是one hot coding)
columns = ['Dayoff', 'Mon', 'Tue', 'Wed', 'Thr', 'Fri', 'Sat', 'Sun',  'YearScaled',
           'MonthScaled', 'Dayscaled', 'NewYear', '3Lock', 'Outbreak','COVID19']
df_multivariate = df[columns]

# 将DataFrame转换为多变量时间序列
ED_covariates = TimeSeries.from_dataframe(df_multivariate)

Hypertuning

In [ ]:
class LossLoggingCallback(Callback):
    def __init__(self):
        super().__init__()
        self.val_losses = []  # To store validation losses
        self.train_losses = []  # To store training losses

    def on_validation_epoch_end(self, trainer, pl_module):
        val_loss = trainer.callback_metrics["val_loss"].item()
        self.val_losses.append(val_loss)
        print(f"Epoch {trainer.current_epoch}: val_loss={val_loss}")
        # Updated report call
        train.report({"loss": val_loss})  # Report the validation loss to Ray Train

    def on_train_epoch_end(self, trainer, pl_module, unused=None):
        if "train_loss" in trainer.callback_metrics:
            train_loss = trainer.callback_metrics["train_loss"].item()
            self.train_losses.append(train_loss)
            print(f"Epoch {trainer.current_epoch}: train_loss={train_loss}")
loss_logging_callback = LossLoggingCallback()
    

In [ ]:
# Create the model using model_args from Ray Tune
def train_model(model_args, callbacks, train, val):
    torch_metrics = MetricCollection([MeanAbsolutePercentageError(), MeanAbsoluteError()])
    
    # Customize the ModelCheckpoint callback
    model_checkpoint_callback = ModelCheckpoint(
        dirpath="checkpoints",
        filename="{epoch}-{val_loss:.2f}",
        every_n_epochs=5,
    )
    
    model = TiDEModel(
        input_chunk_length=30,
        output_chunk_length=1,
        pl_trainer_kwargs={"callbacks": callbacks, "enable_progress_bar": False},
        log_tensorboard=True,
        **model_args)

    model.fit(
    series=[train_series],
    past_covariates=[ED_covariates],
    val_series=[val_series],
    val_past_covariates=[ED_covariates]
    )

In [ ]:
config = {
    'hidden_size': tune.randint(20, 720),
    'lr_scheduler_kwargs': tune.uniform(0, 0.01),
    #'temporal_decoder_hidden': tune.choice([8, 16, 32]),
    "num_encoder_layers": tune.randint(1, 6),
    "num_decoder_layers": tune.randint(1, 6),
    "dropout": tune.uniform(0, 0.8),
}

# earlystopping
my_stopper = EarlyStopping(
    monitor="val_loss",
    patience=5,
    min_delta=0.001,
    mode='min',
)


tune_callback = TuneReportCheckpointCallback(
    {
        "loss": "val_loss",
    },
    on="validation_end",
)



reporter = CLIReporter(
    parameter_columns=list(config.keys()),
    metric_columns=["loss", "MAPE", "training_iteration"],
)

optuna_search = OptunaSearch(metric="loss", mode="min")

In [ ]:
# Run Ray Tune, optimize hyperparameters by minimizing the MAPE on the validation set
num_samples = 30

scheduler = ASHAScheduler(max_t=1000, grace_period=5, reduction_factor=2)

train_fn_with_parameters = tune.with_parameters(
    train_model, callbacks=[my_stopper, tune_callback], train=train_series, val=val_series,
)

analysis = tune.run(
    train_fn_with_parameters,
    #resources_per_trial=resources_per_trial,
    metric="loss",  # any value in TuneReportCallback.
    mode="min",
    config=config,
    num_samples=num_samples,
    search_alg=optuna_search,
    scheduler=scheduler,
    progress_reporter=reporter,
    trial_dirname_creator=lambda trial: str(trial),
    name="tune_darts",
)

print("Best hyperparameters found were: ", analysis.best_config)

In [ ]:
df = analysis.results_df
df

In [ ]:
import plotly.express as px
pd.DataFrame.iteritems = pd.DataFrame.items

df = analysis.results_df

# 假設 df 是你的 DataFrame
fig = px.parallel_coordinates(df, 
                              dimensions=['config/hidden_size', 'config/lr_scheduler_kwargs',
                                          'config/num_encoder_layers','config/num_decoder_layers','config/dropout', 'loss'],
                              color='loss',
                              labels={"config/hidden_size": "Hidden Size",
                                      'config/num_encoder_layers': "Encoder_layers",
                                      'config/num_decoder_layers': "Decoder_layers",
                                      'config/dropout': "Dropout",
                                      "config/lr_scheduler_kwargs": "Learning Rate",
                                      "loss": "MSE",
                                    
                                     },
                              color_continuous_scale=px.colors.diverging.Tealrose,  # 色彩範圍
                              #color_continuous_midpoint=0.0025
                             )  # 中間點，根據數據適當調整

# 設定每個維度的範圍
fig.update_traces(dimensions=[
    dict(range=[min(df['config/hidden_size']), max(df['config/hidden_size'])], label='Hidden layer', values=df['config/hidden_size']),
    dict(range=[min(df['config/lr_scheduler_kwargs']), max(df['config/lr_scheduler_kwargs'])], label='Learning Rate', values=df['config/lr_scheduler_kwargs']),
    dict(range=[min(df['config/dropout']), max(df['config/dropout'])], label='Dropout', values=df['config/dropout']),
    dict(range=[min(df['config/num_encoder_layers']), max(df['config/num_encoder_layers'])], label='Encoder_layers', values=df['config/num_encoder_layers']),
    dict(range=[min(df['config/num_decoder_layers']), max(df['config/num_decoder_layers'])], label='Decoder_layers', values=df['config/num_decoder_layers']),
    dict(range=[min(df['loss']), max(df['loss'])], label='MSE', values=df['loss']),
])

fig.show()

In [ ]:
df.to_csv('C:\\Users\\ian11\\EDtimeseriesForecast\\EDtimeseriesForecast\\Result\\TiDE\\Chiayi\\Hypertuning.csv')